#Skating

## install pyskl 

In [ ]:
!git clone https://github.com/lbaf23/skating-pyskl.git && \
    cd skating-pyskl && \
    pip install -r requirements.txt && \
    pip install -e .

In [ ]:
!pip install imageio==2.4.1

##load dataset
> put `train.pkl` file at google drive dir `skating/`

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## copy dataset

In [5]:
!mkdir skating-pyskl/data
!cp drive/MyDrive/skating/train.pkl skating-pyskl/data

## train

In [ ]:
!cd skating-pyskl && \
    git pull

joint

In [ ]:
!cd skating-pyskl && \
    bash tools/dist_train.sh configs/stgcn/skating/j.py 1

- bone

In [ ]:
!cd skating-pyskl && \
    bash tools/dist_train.sh configs/stgcn/skating/b.py 1